In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch.nn.init as init

In [2]:
# Hyperparams
BATCH_SIZE = 16
LR = 3e-4
NUM_WORKERS = 8
EPOCHS=100
MOMENTUM = 0.9
WEIGHT_DECAY = 1e-4
DEVICE = torch.device('cuda:1')

In [3]:
normMean = [0.485, 0.456, 0.406]
normStd = [0.229, 0.224, 0.225]
normTransform = transforms.Normalize(normMean, normStd)

trainTransform = transforms.Compose([
        transforms.Resize(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normTransform
    ])
testTransform = transforms.Compose([
        transforms.Resize(224),
        transforms.ToTensor(),
        normTransform
    ])

In [4]:
train_dataset = datasets.CIFAR10(root='../DATASETS/classification_datasets', train=True, download=True, transform=trainTransform)
test_dataset = datasets.CIFAR10(root='../DATASETS/classification_datasets', train=False, download=True, transform=testTransform)

train_loader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS, pin_memory=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS, pin_memory=True)

Files already downloaded and verified
Files already downloaded and verified


In [5]:
vgg16 = [64, 64, "M", 128, 128, "M", 256, 256, "M", 512, 512, 512, "M", 512, 512, 512, "M"]
vgg19 = [64, 64, "M", 128, 128, "M", 256, 256, "M", 512, 512, 512, 512, "M", 512, 512, 512, 512, "M"]

In [6]:
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(ConvBlock, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels 

        self.block = nn.Sequential(
            nn.Conv2d(in_channels=in_channels, out_channels = out_channels, kernel_size=3, padding=1, stride=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU()
        )

    def forward(self, x):
        return self.block(x)

In [7]:
class VGGConstructor(nn.Module):
    def __init__(self, in_channels, num_classes, depth):
        super().__init__()
        self.in_channels = in_channels
        self.num_classes = num_classes
        self.depth = depth
        self.conv_layers = self.layers_maker(depth)
        self.classifier = nn.Sequential(
                    nn.Linear(512*7*7, 4096),
                    nn.ReLU(),
                    nn.Dropout(p=0.5),
                    nn.Linear(4096, 4096),
                    nn.ReLU(),
                    nn.Dropout(p=0.5),
                    nn.Linear(4096, num_classes))

    def layers_maker(self, depth):
        in_channels = self.in_channels
        layers = []
        for x in  depth:
            if type(x) is int:
                layer = ConvBlock(in_channels, out_channels=x)
                layers.append(layer)
                in_channels = x
            elif x == "M":
                layer = nn.MaxPool2d(kernel_size=2, stride=2)
                layers.append(layer)

        return nn.Sequential(*layers)
    
    def forward(self, x):
        x = self.conv_layers(x)
        x = torch.flatten(x, start_dim=1)
        x = self.classifier(x)
        return x

In [8]:
model = VGGConstructor(3, 10, vgg19)
model = model.to(DEVICE)

In [9]:
display(model)

VGGConstructor(
  (conv_layers): Sequential(
    (0): ConvBlock(
      (block): Sequential(
        (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
      )
    )
    (1): ConvBlock(
      (block): Sequential(
        (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
      )
    )
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): ConvBlock(
      (block): Sequential(
        (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
      )
    )
    (4): ConvBlock(
      (block): Sequential(
        (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1),

In [10]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer=optimizer, T_max=EPOCHS)

In [11]:
def trainEpoch(device, model, loader, optimizer, criterion):
    loop = tqdm(loader)
    model.train()
    
    losses = []
    correct = 0
    total = 0
    
    for batch_id, (images, labels) in enumerate(loop):
        images, labels=images.to(device), labels.to(device)
        model = model.to(device)
        
        outputs = model(images)
        
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        batch_correct = (predicted==labels).sum().item()
        batch_acc = 100*batch_correct/labels.size(0)
        correct += batch_correct
        
        loss = criterion(outputs, labels)
        losses.append(loss.item())
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        loop.set_postfix(train_loss = loss.item(), train_accuracy = batch_acc)
        
    return np.mean(losses), 100*correct/total

def testEpoch(device, model, loader, criterion):
    model.eval()
    with torch.no_grad():
        losses = []
        correct = 0
        total = 0
        
        loop = tqdm(loader)
        for batch_id, (images, labels) in enumerate(loop):
            images, labels = images.to(device), labels.to(device)
            model = model.to(device)
            
            outputs = model(images)
            
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            batch_correct = (predicted==labels).sum().item()
            batch_acc = 100*batch_correct/labels.size(0)
            correct += batch_correct
            
            loss = criterion(outputs, labels)
            losses.append(loss.item())
            loop.set_postfix(test_loss = loss.item(), test_accuracy = batch_acc)
        return np.mean(losses), 100*correct/total

In [12]:
last_best = 0
train_losses = []
train_acc = []
test_losses = []
test_acc = []
for epoch in range (EPOCHS):
    
    train_loss, train_accuracy = trainEpoch(DEVICE, model, train_loader, optimizer, criterion)
    test_loss, test_accuracy = testEpoch(DEVICE, model, test_loader, criterion)

    train_losses.append(train_loss)
    train_acc.append(train_accuracy)
    test_losses.append(test_loss)
    test_acc.append(test_accuracy)

    scheduler.step()

    if last_best <= test_accuracy:
        torch.save(model.state_dict(), 'vgg19.pt')
        print("Saving new Best Model!")
        last_best = test_accuracy

    print(f"Epoch-{epoch+1} Train Loss: {train_losses[-1]:.4f} Train Accuracy: {train_acc[-1]:.4f}")
    print(f"Epoch-{epoch+1} Test Loss: {test_losses[-1]:.4f} Test Accuracy: {test_acc[-1]:.4f}")

dict = {'train_loss': train_losses, 'train_accuracy': train_acc, 'test_loss': test_losses, 'test_accuracy': test_acc}
df = pd.DataFrame(dict)
df.to_csv('loss_and_acc_vgg19.csv')

100%|██████████| 625/625 [00:31<00:00, 19.88it/s, test_accuracy=43.8, test_loss=1.33]


Saving new Best Model!
Epoch-1 Train Loss: 1.9678 Train Accuracy: 26.2740
Epoch-1 Test Loss: 1.7214 Test Accuracy: 38.2300


100%|██████████| 625/625 [00:26<00:00, 23.88it/s, test_accuracy=68.8, test_loss=1.12] 


Saving new Best Model!
Epoch-2 Train Loss: 1.5329 Train Accuracy: 44.0200
Epoch-2 Test Loss: 1.1855 Test Accuracy: 57.8900


100%|██████████| 625/625 [00:24<00:00, 25.41it/s, test_accuracy=75, test_loss=0.826]  


...

Epoch-98 Train Loss: 0.0040 Train Accuracy: 99.9040
Epoch-98 Test Loss: 1.1211 Test Accuracy: 88.2400


100%|██████████| 625/625 [00:23<00:00, 26.78it/s, test_accuracy=87.5, test_loss=1.31]   


Epoch-99 Train Loss: 0.0046 Train Accuracy: 99.8940
Epoch-99 Test Loss: 1.1554 Test Accuracy: 88.2800


100%|██████████| 625/625 [00:28<00:00, 21.87it/s, test_accuracy=93.8, test_loss=0.0505] 

Epoch-100 Train Loss: 0.0023 Train Accuracy: 99.9360
Epoch-100 Test Loss: 1.1507 Test Accuracy: 88.2700
